# 基于MindSpore构造平方损失函数

## 导入所需库和函数

In [1]:
from mindspore import context
import numpy as np
from mindspore import dataset as ds
from mindspore import nn, Tensor, Model
import time
from mindspore.train.callback import Callback, LossMonitor
import mindspore as ms
import mindspore.ops as ops
from mindspore.nn import L1Loss
ms.common.set_seed(0)


## 定义数据集

In [2]:
def get_data(num, a=2.0, b=3.0, c=5.0):
    for _ in range(num):
        x = np.random.uniform(-1.0, 1.0)
        y = np.random.uniform(-1.0, 1.0)
        noise = np.random.normal(0, 0.03)
        z = a * x ** 2 + b * y ** 3 + c + noise
        yield np.array([[x**2], [y**3]],dtype=np.float32).reshape(1,2), np.array([z]).astype(np.float32)
 
def create_dataset(num_data, batch_size=16, repeat_size=1):
    input_data = ds.GeneratorDataset(list(get_data(num_data)), column_names=['xy','z'])
    input_data = input_data.batch(batch_size)
    input_data = input_data.repeat(repeat_size)
    return input_data
 
data_number = 160
batch_number = 10
repeat_number = 10
 
ds_train = create_dataset(data_number, batch_size=batch_number, repeat_size=repeat_number)

## 定义一个简单的模型

In [3]:
class LinearNet(nn.Cell):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(2, 1, 0.02, 0.02)
 
    def construct(self, x):
        x = self.fc(x)
        return x


## 查看模型中参数维度

In [4]:
start_time = time.time()
net = LinearNet()
model_params = net.trainable_params()
print ('Param Shape is: {}'.format(len(model_params)))
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())


Param Shape is: 2
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[0.02 0.02]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [0.02]


## 重写损失函数，自定义平方损失函数

In [5]:
class quadratic_loss(L1Loss):
    def __init__(self, reduction="mean"):
        super(quadratic_loss, self).__init__(reduction)
        self.square = ops.Square()
 
    def construct(self, base, target):
        x = 0.5 * self.square(base - target)
        return self.get_loss(x)
 
user_loss = quadratic_loss()


## 训练

In [6]:
optim = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.6)
model = Model(net, user_loss, optim)
 
epoch = 1
model.train(epoch, ds_train, callbacks=[LossMonitor(10)], dataset_sink_mode=True)
 
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())
 
print ('The total time cost is: {}s'.format(time.time() - start_time))


epoch: 1 step: 160, loss is 1.2997653484344482
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[1.3762274  0.01708973]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [5.0189986]
The total time cost is: 20.236934423446655s
